In [7]:
from stocktrends import Renko
import yfinance as yf

tickers=['AMZN','GOOG','TSLA']
ohlcv_data={}
hour_data={}
renko_data={}
for ticker in tickers:
    temp=yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how='any',inplace=True)
    ohlcv_data[ticker]=temp
    temp=yf.download(ticker,period='1y',interval='1h')
    temp.dropna(how='any',inplace=True)
    hour_data[ticker]=temp

def ATR(DF,n=14):
    df=DF.copy()
    df['H-L']=df['High']-df['Low']
    df['H-PC']=df['High']-df['Adj Close'].shift(1)
    df['L-PC']=df['Low']-df['Adj Close'].shift(1)
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR']=df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
def renko_DF(DF, hourly_df):
    df = DF.copy()
    df.drop('Close',axis=1,inplace=True)
    df.reset_index(inplace=True)
    df.columns = ['date','open','high','low','close','volume']
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data()
    return renko_df

In [9]:
for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])

In [11]:
renko_data['AMZN']

,date,open,high,low,close,uptrend
0,2023-08-07 09:30:00-04:00,135.0,138.0,135.0,138.0,True
1,2023-08-07 12:25:00-04:00,138.0,141.0,138.0,141.0,True
2,2023-08-17 09:50:00-04:00,138.0,138.0,135.0,135.0,False
3,2023-08-18 09:30:00-04:00,135.0,135.0,132.0,132.0,False
4,2023-08-31 10:00:00-04:00,135.0,138.0,135.0,138.0,True
